In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
!pip install botorch


import json

from pathlib import Path
import torch
import botorch
from botorch.models import SingleTaskGP
from botorch.utils import standardize
from botorch.acquisition import UpperConfidenceBound
from botorch.acquisition import ExpectedImprovement
from botorch.sampling import SobolQMCNormalSampler
from botorch.acquisition import qNoisyExpectedImprovement
from botorch.optim import optimize_acqf

from gpytorch.mlls.exact_marginal_log_likelihood import ExactMarginalLogLikelihood
from botorch import fit_gpytorch_model
from botorch.acquisition.monte_carlo import qExpectedImprovement



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 530.3/530.3 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 732.5/732.5 kB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.2/255.2 kB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.7/156.7 kB 16.8 MB/s eta 0:00:00


In [ ]:


data_path_bayesian_opti = Path("/content/gdrive/MyDrive/Thesis_deeplearning_models/")

filename_training_bayesian_opti = data_path_bayesian_opti / "training_data_opti.txt"

# Read the contents of the file
with open(filename_training_bayesian_opti, 'r') as file:
    content = file.read()

    # Parse the content as JSON
    training_data_bayesian_opti = json.loads(content)

import numpy as np




original_bound_values = {
    'optimizer':['SGD', 'Adam'],
    'architecture_used': ['DeepLabV3'],
    'network': ['resnet18'],
    'batch_size': [4,8,16],
    'momentum': [0.9],
    'loss_function': ['CrossEntropyLoss']

}


transformed_bound_values = {}

training_data_bayesian_opti_modified = training_data_bayesian_opti.copy()

valid_rows = []


"""
Transform the original values into values that can be processed by the model
"""
for row_training_data_modified in training_data_bayesian_opti_modified:
    ok = True
    for key in original_bound_values:
        features_table = original_bound_values[key]
        # Create a dictionary to map batch sizes to integer values
        size_mapping = {size: idx for idx, size in enumerate(features_table)}

        # Perform integer encoding on the dataset
        encoded_data = [size_mapping[features] for features in features_table]
        tuple_encoded_data = (encoded_data[0], encoded_data[-1])
        # Updates the values to numerical ones
        transformed_bound_values[key] = tuple_encoded_data

        if row_training_data_modified[key] in size_mapping:
            row_training_data_modified[key] = size_mapping[row_training_data_modified[key]]
        else:
            ok = False


    if(ok == True):
      valid_rows.append(row_training_data_modified)


training_data_bayesian_opti_modified = valid_rows



countb = 0
for features_row_modified in training_data_bayesian_opti_modified:
    countb+=1
print("Number of valid rows : " + str(countb))


# Define the bounds for each hyperparameter
bounds = [
    (1,30),                                             # What is the best epoch between 1 and 30? We ran experiments between 10 and 30.
    transformed_bound_values['optimizer'],              # optimizer: 0 for 'SGD', 1 for 'Adam'
    transformed_bound_values['architecture_used'],      # architecture_used: 0 for 'DeepLabV3'
    transformed_bound_values['network'],                # network: 0 for 'resnet18'
    transformed_bound_values['batch_size'],             # batch_size: 0 for 4, 1 for 8, 2 for 16
    (0.0001, 0.9),                                      # learning_rate
    transformed_bound_values['momentum'],               # momentum: 0.9 (fixed value)
    (0.0001, 0.9),                                      # weight_decay
    transformed_bound_values['loss_function'],          # loss_function: 0 for 'CrossEntropyLoss', 1 for 'Focal Loss', 2 'Dice Loss'
]

print(bounds)


X = torch.tensor([[features_row['epoch'], features_row['optimizer'], features_row['architecture_used'],
                   features_row['network'], features_row['batch_size'], features_row['learning_rate'],
                                      features_row['momentum'], features_row['weight_decay'], features_row['loss_function']]
                                                        for features_row in training_data_bayesian_opti_modified], dtype=torch.double)



y_ibg = torch.tensor([[features_row['test_miou_acc_ibg']] for features_row in training_data_bayesian_opti_modified])



X = torch.tensor(X, dtype=torch.double)
y_ibg = torch.tensor(y_ibg, dtype=torch.double)

best_observed_y_ibg = y_ibg.max().item()
print("best observed value for y_ibg " + str(best_observed_y_ibg))

"""

"""

bounds = np.array(bounds)
# Reshape the bounds to a 2 x 9 tensor
bounds = bounds.T
# Convert the bounds NumPy array to a PyTorch tensor
bounds = torch.tensor(bounds)

# SingleTaskGP model
model_ibg = SingleTaskGP(X, y_ibg)

# Define the acquisition function to maximize (e.g., Upper Confidence Bound)
#acq_func = UpperConfidenceBound(model_ibg, beta=0.1)


mll = ExactMarginalLogLikelihood(model_ibg.likelihood,model_ibg)
fit_gpytorch_model(mll)

acq_func = qExpectedImprovement(model_ibg, best_f=best_observed_y_ibg)


# Optimize the hyperparameters based on the defined acquisition function
best_hyperparameters_ibg, _ = optimize_acqf(
    acq_function=acq_func,
    bounds=bounds,
    q=1,
    num_restarts=100,
    raw_samples=200,
)


print(best_hyperparameters_ibg)

# Change the tensor to a list
best_operational_hyperparameters = best_hyperparameters_ibg.tolist()[0]

print(best_operational_hyperparameters)


"""
Reverse mapping function, takes the numerical value as well as the hyperparameter concerned in input,
and returns the original value
"""
def reverse_mapping(hyperparameter, value_to_reverse):
    features_table = original_bound_values[hyperparameter]
    # Create a dictionary to map batch sizes to integer values
    size_mapping = {size: idx for idx, size in enumerate(features_table)}

    # Reverse the key-value pairs in size_mapping
    reversed_size_mapping = {idx: size for size, idx in size_mapping.items()}

    value_to_reverse = round(value_to_reverse)
    return reversed_size_mapping[value_to_reverse]


best_operational_hyperparameters = {
    'epoch': round(best_operational_hyperparameters[0]),
    'optimizer': reverse_mapping("optimizer",best_operational_hyperparameters[1]),
    'architecture_used': reverse_mapping("architecture_used",best_operational_hyperparameters[2]),
    'network': reverse_mapping("network",best_operational_hyperparameters[3]),
    'batch_size': reverse_mapping("batch_size",best_operational_hyperparameters[4]),
    'learning_rate': best_operational_hyperparameters[5],
    'momentum': reverse_mapping("momentum",best_operational_hyperparameters[6]),
    'weight_decay': best_operational_hyperparameters[7],
    'loss_function': reverse_mapping("loss_function",best_operational_hyperparameters[8])

}

print(best_operational_hyperparameters)


del(X)
del(y_ibg)
del(model_ibg)
del(acq_func)
del(best_hyperparameters_ibg)
del(best_operational_hyperparameters)




{'version_experiment': 2003, 'epoch': 1, 'test_miou_acc_ibg': 0.0, 'optimizer': 0, 'architecture_used': 0, 'network': 0, 'batch_size': 1, 'learning_rate': 0.8101972717544853, 'momentum': 0, 'weight_decay': 0.5128159537541757, 'loss_function': 0}
{'version_experiment': 2003, 'epoch': 2, 'test_miou_acc_ibg': 0.044774421490728855, 'optimizer': 0, 'architecture_used': 0, 'network': 0, 'batch_size': 1, 'learning_rate': 0.8101972717544853, 'momentum': 0, 'weight_decay': 0.5128159537541757, 'loss_function': 0}
{'version_experiment': 2003, 'epoch': 3, 'test_miou_acc_ibg': 0.0, 'optimizer': 0, 'architecture_used': 0, 'network': 0, 'batch_size': 1, 'learning_rate': 0.8101972717544853, 'momentum': 0, 'weight_decay': 0.5128159537541757, 'loss_function': 0}
{'version_experiment': 2003, 'epoch': 4, 'test_miou_acc_ibg': 0.0, 'optimizer': 0, 'architecture_used': 0, 'network': 0, 'batch_size': 1, 'learning_rate': 0.8101972717544853, 'momentum': 0, 'weight_decay': 0.5128159537541757, 'loss_function': 0}

<ipython-input-21-288305f7ba53>:102: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = torch.tensor(X, dtype=torch.double)
<ipython-input-21-288305f7ba53>:103: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_ibg = torch.tensor(y_ibg, dtype=torch.double)
/usr/local/lib/python3.10/dist-packages/botorch/models/utils/assorted.py:173: InputDataWarning: Input data is not contained to the unit cube. Please consider min-max scaling the input data.
  warnings.warn(msg, InputDataWarning)
/usr/local/lib/python3.10/dist-packages/botorch/models/utils/assorted.py:201: InputDataWarning: Input data is not standardized. Please consider scaling the input to zero mean and unit variance.
  warnings.warn(msg

tensor([[8.7924e+00, 7.1194e-20, 0.0000e+00, 0.0000e+00, 1.0000e+00, 8.1019e-01,
         0.0000e+00, 5.1282e-01, 0.0000e+00]], dtype=torch.float64)
[8.792405449853957, 7.119391844327822e-20, 0.0, 0.0, 1.0000025635008276, 0.8101881214852589, 0.0, 0.5128153937755331, 0.0]
{'epoch': 9, 'optimizer': 'SGD', 'architecture_used': 'DeepLabV3', 'network': 'resnet18', 'batch_size': 8, 'learning_rate': 0.8101881214852589, 'momentum': 0.9, 'weight_decay': 0.5128153937755331, 'loss_function': 'CrossEntropyLoss'}
